# PeRCNN求解3D 反应扩散方程

## 概述

近日，华为与中国人民大学孙浩教授团队合作，基于昇腾AI基础软硬件平台与昇思
MindSpore AI框架提出了一种[物理编码递归卷积神经网络（Physics-encoded Recurrent Convolutional Neural Network， PeRCNN）](https://www.nature.com/articles/s42256-023-00685-7)。相较于物理信息神经网络、ConvLSTM、PDE-NET等方法，模型泛化性和抗噪性明显提升，长期推理精度提升了
10倍以上，在航空航天、船舶制造、气象预报等领域拥有广阔的应用前景，目前该成果已在 nature machine intelligence 上发表。

## 问题描述

反应扩散方程（reaction-diffusion equation）是非常重要且应用广泛的一类偏微分方程，它描述了物理学中的种种现象，也在化学反应中被广泛使用。

## 控制方程

在本研究中，反应扩散方程的形式为：

$$
u_t = \mu_u \Delta u - u{v*2} + F(1-v)
$$

$$
v_t = \mu_v \Delta v + u{v*2} + (F+\kappa)v
$$

其中，

$$
\mu_v = 0.1, \mu_u = 0.2, F = 0.025, \kappa = 0.055
$$

在本案例中，拟在$ \Omega \times \tau = {[-50,50]}^3 \times [0,500] $ 的物理域中求解100个时间步的流场演化（时间步长为0.5s），初始条件经历了高斯加噪，采取周期性边界条件。


## 技术路径

MindFlow求解该问题的具体流程如下：

1. 优化器
2. 构建模型
3. 模型训练
4. 模型推理及可视化。

In [1]:
import os
import sys
import time

import numpy as np


In [2]:
from mindspore import context, jit, nn, ops, save_checkpoint, set_seed
import mindspore.common.dtype as mstype
from mindflow.utils import load_yaml_config, print_log
from src import RecurrentCnn, post_process, Trainer, UpScaler, count_params


In [3]:
set_seed(123456)
np.random.seed(123456)
context.set_context(mode=context.GRAPH_MODE, device_target="GPU", device_id=0)

## 优化器和单步训练

In [4]:
def train_stage(trainer, stage, config, ckpt_dir, use_ascend):
    """train stage"""
    if use_ascend:
        from mindspore.amp import DynamicLossScaler, all_finite
        loss_scaler = DynamicLossScaler(2**10, 2, 100)

    if 'milestone_num' in config.keys():
        milestone = list([(config['epochs']//config['milestone_num'])*(i + 1)
                          for i in range(config['milestone_num'])])
        learning_rate = config['learning_rate']
        lr = float(config['learning_rate'])*np.array(list([config['gamma']
                                                           ** i for i in range(config['milestone_num'])]))
        learning_rate = nn.piecewise_constant_lr(milestone, list(lr))
    else:
        learning_rate = config['learning_rate']

    if stage == 'pretrain':
        params = trainer.upconv.trainable_params()
    else:
        params = trainer.upconv.trainable_params() + trainer.recurrent_cnn.trainable_params()

    optimizer = nn.Adam(params, learning_rate=learning_rate)

    def forward_fn():
        if stage == 'pretrain':
            loss = trainer.get_ic_loss()
        else:
            loss = trainer.get_loss()
        if use_ascend:
            loss = loss_scaler.scale(loss)
        return loss

    if stage == 'pretrain':
        grad_fn = ops.value_and_grad(forward_fn, None, params, has_aux=False)
    else:
        grad_fn = ops.value_and_grad(forward_fn, None, params, has_aux=True)

    @jit
    def train_step():
        loss, grads = grad_fn()
        if use_ascend:
            loss = loss_scaler.unscale(loss)
            is_finite = all_finite(grads)
            if is_finite:
                grads = loss_scaler.unscale(grads)
                loss = ops.depend(loss, optimizer(grads))
            loss_scaler.adjust(is_finite)
        else:
            loss = ops.depend(loss, optimizer(grads))
        return loss

    best_loss = sys.maxsize
    for epoch in range(1, 1 + config['epochs']):
        time_beg = time.time()
        trainer.upconv.set_train(True)
        trainer.recurrent_cnn.set_train(True)
        if stage == 'pretrain':
            step_train_loss = train_step()
            print_log(
                f"epoch: {epoch} train loss: {step_train_loss} epoch time: {(time.time() - time_beg) :.3f} s")
        else:
            if epoch == 3800:
                break
            epoch_loss, loss_data, loss_ic, loss_phy, loss_valid = train_step()
            print_log(f"epoch: {epoch} train loss: {epoch_loss} ic_loss: {loss_ic} data_loss: {loss_data} \
 phy_loss: {loss_phy}  valid_loss: {loss_valid} epoch time: {(time.time() - time_beg): .3f} s")
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                print_log('best loss', best_loss, 'save model')
                save_checkpoint(trainer.upconv, os.path.join(ckpt_dir, "train_upconv.ckpt"))
                save_checkpoint(trainer.recurrent_cnn,
                                os.path.join(ckpt_dir, "train_recurrent_cnn.ckpt"))


## 构建模型

PeRCNN要构建两个网络，一个是做上采样的UpSclaer，一个是作为主体的recurrent CNN。

In [5]:
def train():
    """train"""
    rd_config = load_yaml_config('./configs/percnn_3d_rd.yaml')
    data_config = rd_config['data']
    optim_config = rd_config['optimizer']
    summary_config = rd_config['summary']
    model_config = rd_config['model']

    use_ascend = context.get_context(attr_key='device_target') == "Ascend"
    print_log(f"use_ascend: {use_ascend}")

    if use_ascend:
        compute_dtype = mstype.float16
    else:
        compute_dtype = mstype.float32

    upconv_config = model_config['upconv']
    upconv = UpScaler(in_channels=upconv_config['in_channel'],
                      out_channels=upconv_config['out_channel'],
                      hidden_channels=upconv_config['hidden_channel'],
                      kernel_size=upconv_config['kernel_size'],
                      stride=upconv_config['stride'],
                      has_bais=True)

    if use_ascend:
        from mindspore.amp import auto_mixed_precision
        auto_mixed_precision(upconv, 'O1')

    rcnn_config = model_config['rcnn']
    recurrent_cnn = RecurrentCnn(input_channels=rcnn_config['in_channel'],
                                 hidden_channels=rcnn_config['hidden_channel'],
                                 kernel_size=rcnn_config['kernel_size'],
                                 stride=rcnn_config['stride'],
                                 compute_dtype=compute_dtype)

    percnn_trainer = Trainer(upconv=upconv,
                             recurrent_cnn=recurrent_cnn,
                             timesteps_for_train=data_config['rollout_steps'],
                             dx=data_config['dx'],
                             grid_size=data_config['grid_size'],
                             dt=data_config['dt'],
                             mu=data_config['mu'],
                             data_path=data_config['data_path'],
                             compute_dtype=compute_dtype)

    total_params = int(count_params(upconv.trainable_params()) +
                       count_params(recurrent_cnn.trainable_params()))
    print(f"There are {total_params} parameters")

    ckpt_dir = summary_config["ckpt_dir"]
    fig_path = summary_config["fig_save_path"]
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    train_stage(percnn_trainer, 'pretrain',
                optim_config['pretrain'], ckpt_dir, use_ascend)
    train_stage(percnn_trainer, 'finetune',
                optim_config['finetune'], ckpt_dir, use_ascend)

    output = percnn_trainer.get_output(100).asnumpy()
    output = np.transpose(output, (1, 0, 2, 3, 4))[:, :-1:10]

    print('output shape is ', output.shape)
    for i in range(0, 10, 2):
        post_process(output[0, i], fig_path, is_u=True, num=i)


## 模型训练

使用**MindSpore >= 2.0.0**的版本，可以使用函数式编程范式训练神经网络。

In [ ]:
train()

use_ascend: False
shape of uv is  (3001, 2, 48, 48, 48)
shape of ic is  (1, 2, 48, 48, 48)
shape of init_state_low is  (1, 2, 24, 24, 24)
There are 10078 parameters


[ERROR] CORE(60439,7ff577089200,python):2023-10-19-18:58:20.403.131 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_60439/47002643.py]
[ERROR] CORE(60439,7ff577089200,python):2023-10-19-18:58:20.403.227 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_60439/47002643.py]


epoch: 1 train loss: 0.160835 epoch time: 5.545 s
epoch: 2 train loss: 104.36749 epoch time: 0.010 s
epoch: 3 train loss: 4.3207517 epoch time: 0.009 s
epoch: 4 train loss: 8.491383 epoch time: 0.009 s
epoch: 5 train loss: 23.683647 epoch time: 0.009 s
epoch: 6 train loss: 23.857117 epoch time: 0.010 s
epoch: 7 train loss: 16.037672 epoch time: 0.010 s
epoch: 8 train loss: 8.406443 epoch time: 0.009 s
epoch: 9 train loss: 3.527469 epoch time: 0.020 s
epoch: 10 train loss: 1.0823832 epoch time: 0.009 s
epoch: 11 train loss: 0.18675002 epoch time: 0.018 s
epoch: 12 train loss: 0.13879554 epoch time: 0.019 s
epoch: 13 train loss: 0.4895127 epoch time: 0.013 s
epoch: 14 train loss: 0.9677096 epoch time: 0.008 s
epoch: 15 train loss: 1.4145753 epoch time: 0.012 s
epoch: 16 train loss: 1.7454404 epoch time: 0.019 s
epoch: 17 train loss: 1.9276694 epoch time: 0.022 s
epoch: 18 train loss: 1.9647502 epoch time: 0.009 s
epoch: 19 train loss: 1.8823279 epoch time: 0.020 s
epoch: 20 train loss: 1

epoch: 9977 train loss: 8.792636e-05 epoch time: 0.007 s
epoch: 9978 train loss: 8.811904e-05 epoch time: 0.008 s
epoch: 9979 train loss: 8.8046014e-05 epoch time: 0.008 s
epoch: 9980 train loss: 8.781863e-05 epoch time: 0.007 s
epoch: 9981 train loss: 8.778041e-05 epoch time: 0.007 s
epoch: 9982 train loss: 8.790506e-05 epoch time: 0.007 s
epoch: 9983 train loss: 8.791076e-05 epoch time: 0.008 s
epoch: 9984 train loss: 8.77632e-05 epoch time: 0.008 s
epoch: 9985 train loss: 8.768207e-05 epoch time: 0.008 s
epoch: 9986 train loss: 8.7740336e-05 epoch time: 0.008 s
epoch: 9987 train loss: 8.777662e-05 epoch time: 0.008 s
epoch: 9988 train loss: 8.769672e-05 epoch time: 0.008 s
epoch: 9989 train loss: 8.760943e-05 epoch time: 0.008 s
epoch: 9990 train loss: 8.7615306e-05 epoch time: 0.008 s
epoch: 9991 train loss: 8.76504e-05 epoch time: 0.008 s
epoch: 9992 train loss: 8.761823e-05 epoch time: 0.008 s
epoch: 9993 train loss: 8.7546505e-05 epoch time: 0.008 s
epoch: 9994 train loss: 8.751

epoch: 1 train loss: 61.754555 ic_loss: 8.7413886e-05 data_loss: 6.1754117  phy_loss: 2.6047118  valid_loss: 7.221066 truth_loss: 2.7125626 epoch time:  138.495 s
best loss 61.754555 save model
epoch: 2 train loss: 54.79151 ic_loss: 0.32984126 data_loss: 5.3142304  phy_loss: 52.50226  valid_loss: 6.812231 truth_loss: 2.7744124 epoch time:  1.342 s
best loss 54.79151 save model
epoch: 3 train loss: 46.904842 ic_loss: 0.12049961 data_loss: 4.6302347  phy_loss: 32.494545  valid_loss: 5.953037 truth_loss: 2.579268 epoch time:  1.262 s
best loss 46.904842 save model
epoch: 4 train loss: 40.674736 ic_loss: 0.031907484 data_loss: 4.05152  phy_loss: 11.360751  valid_loss: 5.08032 truth_loss: 2.3503494 epoch time:  1.233 s
best loss 40.674736 save model
epoch: 5 train loss: 36.910408 ic_loss: 0.10554239 data_loss: 3.6382694  phy_loss: 3.5776496  valid_loss: 4.4271708 truth_loss: 2.1671412 epoch time:  1.315 s
best loss 36.910408 save model
epoch: 6 train loss: 33.767193 ic_loss: 0.14396289 data

epoch: 1640 train loss: 0.094870105 ic_loss: 0.0006555757 data_loss: 0.009159223  phy_loss: 0.000713372  valid_loss: 0.012182931 truth_loss: 0.16177362 epoch time:  1.289 s
best loss 0.094870105 save model
epoch: 1641 train loss: 0.09474868 ic_loss: 0.00065547746 data_loss: 0.00914713  phy_loss: 0.00071231654  valid_loss: 0.01216803 truth_loss: 0.16169967 epoch time:  1.259 s
best loss 0.09474868 save model
epoch: 1642 train loss: 0.09462735 ic_loss: 0.0006553787 data_loss: 0.009135046  phy_loss: 0.00071125705  valid_loss: 0.012153144 truth_loss: 0.16162594 epoch time:  1.310 s
best loss 0.09462735 save model
epoch: 1643 train loss: 0.094506115 ic_loss: 0.000655279 data_loss: 0.009122972  phy_loss: 0.00071020663  valid_loss: 0.01213827 truth_loss: 0.16155209 epoch time:  1.379 s
best loss 0.094506115 save model
epoch: 1644 train loss: 0.094384976 ic_loss: 0.0006551788 data_loss: 0.009110908  phy_loss: 0.0007091502  valid_loss: 0.012123411 truth_loss: 0.16147849 epoch time:  1.375 s
bes

## 模型推理及可视化

完成训练后，下图展示了预测结果和真实标签的对比情况。

![](./images/result.jpg)